In [101]:
import pandas as pd
import numpy as np
import pprint
from sklearn.preprocessing import OrdinalEncoder
eps = np.finfo(float).eps
print(eps)

2.220446049250313e-16


In [102]:
data = pd.read_csv("lab01_dataset_1.csv")
data.head()

,Mood,Effort,Score,Output
0,Happy,Low,35,Yes
1,Happy,Medium,91,No
2,Happy,High,52,No
3,Neutral,Low,83,No
4,Neutral,Medium,48,No


In [103]:
data.tail()

,Mood,Effort,Score,Output
14,Neutral,High,50,No
15,Sad,Low,69,No
16,Sad,Medium,70,Yes
17,Sad,High,95,No
18,Sad,Low,80,Yes


In [104]:
print(data.columns)
data.shape

Index(['Mood', 'Effort', 'Score', 'Output'], dtype='object')


(19, 4)

In [105]:
data.dtypes

Mood      object
Effort    object
Score      int64
Output    object
dtype: object

In [106]:
# sort the data by score
def sortDataFrameByScore(data):
    data = data.sort_values(by='Score', ignore_index=True)
    data.head()
    return data

In [107]:
data = sortDataFrameByScore(data)
data

,Mood,Effort,Score,Output
0,Happy,Low,35,Yes
1,Happy,High,39,Yes
2,Sad,Low,40,Yes
3,Happy,Low,44,Yes
4,Neutral,Medium,48,No
5,Neutral,High,50,No
6,Happy,High,52,No
7,Neutral,High,61,No
8,Happy,Low,66,No
9,Sad,Low,69,No


In [108]:
def splitPoint(data):
    attributeDataFrame = pd.DataFrame({ 'Score' : data['Score'], 'Output' : data['Output']})
    attributeDataFrame.reindex(np.arange(len(attributeDataFrame['Score'].index)))
    print(attributeDataFrame)

    outputvalue = attributeDataFrame['Output'][0]

    splitPointDict = {}

    for i in range(1, len(attributeDataFrame['Score'])):
        if(attributeDataFrame['Output'][i] != outputvalue):
            # (i-1) (i) mean
            mean = (attributeDataFrame['Score'][i-1] + attributeDataFrame['Score'][i])/2
            splitPointDict[i-1] = mean
            outputvalue = attributeDataFrame['Output'][i]
        
    pprint.pprint(splitPointDict)
    return attributeDataFrame, splitPointDict
attributeDataFrame, splitPoint = splitPoint(data)

    Score Output
0      35    Yes
1      39    Yes
2      40    Yes
3      44    Yes
4      48     No
5      50     No
6      52     No
7      61     No
8      66     No
9      69     No
10     70    Yes
11     73    Yes
12     75    Yes
13     80    Yes
14     83     No
15     86     No
16     91     No
17     95     No
18     98     No
{3: 46.0, 9: 69.5, 13: 81.5}


In [109]:
for i in splitPoint:
    split = splitPoint[i]
    print(splitPoint[i])
    ltYes = 0
    ltNo = 0
    gtYes = 0
    gtNo = 0

    for j in range(0, len(attributeDataFrame['Score'])):
        if attributeDataFrame['Score'][j] < split and attributeDataFrame['Output'][j] == 'Yes':
            ltYes+=1
        if attributeDataFrame['Score'][j] < split and attributeDataFrame['Output'][j] == 'No':
            ltNo+=1

        if attributeDataFrame['Score'][j] >= split and attributeDataFrame['Output'][j] == 'Yes':
            gtYes+=1
        if attributeDataFrame['Score'][j] >= split and attributeDataFrame['Output'][j] == 'No':
            gtNo+=1

    print(ltYes , " | " , ltNo  , " | " , gtYes , " | " , gtNo)

46.0
4  |  0  |  4  |  11
69.5
4  |  6  |  4  |  5
81.5
8  |  6  |  0  |  5


In [110]:
columnsList = {}

for i in splitPoint:
    split = splitPoint[i]
    # "Score_" + str(split)
    scoreList=[]
    for j in range(0, len(attributeDataFrame['Score'])):
        if attributeDataFrame['Score'][j] < split:
            scoreList.append(1)
        else: 
            scoreList.append(0)

    columnsList["Score_" + str(split)] = scoreList

columnsList

{'Score_46.0': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'Score_69.5': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'Score_81.5': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]}

In [111]:
data['Score_46.0'] = columnsList['Score_46.0']
data['Score_69.5'] = columnsList['Score_69.5']
data['Score_81.5'] = columnsList['Score_81.5']

In [112]:
# featire Score is dropped
data = pd.DataFrame({
    'Mood' : data['Mood'], 
    'Effort' : data['Effort'], 
    'Score_46.0' : data['Score_46.0'], 
    'Score_69.5' : data['Score_69.5'], 
    'Score_81.5' : data['Score_81.5'],
    'Output' : data['Output']
})

data.head()

,Mood,Effort,Score_46.0,Score_69.5,Score_81.5,Output
0,Happy,Low,1,1,1,Yes
1,Happy,High,1,1,1,Yes
2,Sad,Low,1,1,1,Yes
3,Happy,Low,1,1,1,Yes
4,Neutral,Medium,0,1,1,No


In [113]:

moodUniqueValues = list(np.asarray(data['Mood'].unique()))
encoderMood = OrdinalEncoder(categories=[list(moodUniqueValues)])
data['Mood'] = encoderMood.fit_transform(data[['Mood']])

effortUniqueValues = list(np.asarray(data['Effort'].unique()))
encoderEffort = OrdinalEncoder(categories=[list(effortUniqueValues)])
data['Effort'] = encoderEffort.fit_transform(data[['Effort']])

outputUniqueValues = list(np.asarray(data['Output'].unique()))
encoderOutput = OrdinalEncoder(categories=[list(outputUniqueValues)])
data['Output'] = encoderOutput.fit_transform(data[['Output']])


data.head()

,Mood,Effort,Score_46.0,Score_69.5,Score_81.5,Output
0,0.0,0.0,1,1,1,0.0
1,0.0,1.0,1,1,1,0.0
2,1.0,0.0,1,1,1,0.0
3,0.0,0.0,1,1,1,0.0
4,2.0,2.0,0,1,1,1.0


#### 3. Your function my_ID3() should operate in a manner such that after every round of decision making, it will output the attributes and its associated gain, with a message stating “Attribute X with Gain = Y is chosen as the decision attribute”. Once your function completes, it should output the decision tree. The representation of the decision tree is upto you. You can choose either a textual representation or a graphical one; either is fine. (10 marks)

In [123]:
def calculateEntropy(data):
    targetClass = data.keys()[-1]
    entropyValue = 0
    targetClassUnique = data[targetClass].unique()

    for item in targetClassUnique:
        proportion = data[targetClass].value_counts()[item]/ len (data[targetClass])
        entropyValue += -proportion * np.log2(proportion)

    return entropyValue

def getEntropyAttribute(data,attribute):
    targetClass = data.keys()[-1]
    targetUniqueValues = data[targetClass].unique()
    
    attributeUniqueValue = data[attribute].unique()
    entropyValue = 0

    for unique in attributeUniqueValue:
        entropy = 0
        for target in targetUniqueValues:
            num = len(data[attribute][data[attribute] == unique][data[targetClass] == target])
            den = len(data[attribute][data[attribute] == unique])

            fraction = num/ (den + eps)
            entropy += -fraction * np.log2(fraction + eps)
        
        fraction2=den/len(data)
        entropyValue += -fraction2 * entropy

    return abs(entropyValue)

def builtClassifier(data):
    informationGain = []

    for key in data.keys()[:-1]:
        print(key, "", calculateEntropy(data) - getEntropyAttribute(data, key) )
        informationGain.append(calculateEntropy(data) - getEntropyAttribute(data, key))

    print(informationGain)
    return data.keys()[:-1][np.argmax(informationGain)] 

In [125]:
def getSubtable(data, feature, value):
    return data[data[feature] == value].reset_index(drop=True)

def my_ID3(data, tree=None):
    targetClass = data.keys()[-1]
    attribute = builtClassifier(data)

    attributeUnique = np.unique(data[attribute])

    if tree is None:
        tree = {}
        tree[attribute] = {}


    for unique in attributeUnique:
        subTable = getSubtable(data, attribute, unique)

        targetValues, valueCounts = np.unique(subTable[targetClass], return_counts=True)

        if len(valueCounts) == 1:
            tree[attribute][unique] = targetValues[0]
        else:
            tree[attribute][unique] = my_ID3(subTable)
    
    return tree

decisionTree = my_ID3(data)
print(decisionTree)

Mood  0.06600213534218613
Effort  0.03898568365917021
Score_46.0  0.3214349379631768
Score_69.5  0.0014570772277464616
Score_81.5  0.2559832129441235
[0.06600213534218613, 0.03898568365917021, 0.3214349379631768, 0.0014570772277464616, 0.2559832129441235]
Mood  0.19599804364538032
Effort  0.012338876834482537
Score_46.0  6.661338147750939e-16
Score_69.5  0.24199510603823438
Score_81.5  0.18934034563805535
[0.19599804364538032, 0.012338876834482537, 6.661338147750939e-16, 0.24199510603823438, 0.18934034563805535]
Mood  0.2294368406967402
Effort  0.0072146184745179864
Score_46.0  6.661338147750939e-16
Score_69.5  6.661338147750939e-16
Score_81.5  0.9910760598382219
[0.2294368406967402, 0.0072146184745179864, 6.661338147750939e-16, 6.661338147750939e-16, 0.9910760598382219]
{'Score_46.0': {0: {'Score_69.5': {0: {'Score_81.5': {0: 1.0, 1: 0.0}}, 1: 1.0}}, 1: 0.0}}
